Modello giocattolo per testare attivazioni

MODELING MODULES

In [1]:
from mesa import Agent, Model

from mesa.time import RandomActivation
from mesa.time import SimultaneousActivation
from mesa.time import StagedActivation #dividere le azioni di un agenti (probabilmente inutile per noi)
from mesa.time import BaseScheduler

from mesa.space import MultiGrid


class MyAgent(Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name

    def step(self):
        print("{} activated".format(self.name))
        # Whatever else the agent does when activated

class MyModel(Model):      
    
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = RandomActivation(self) #choose a schedular method
        self.grid = MultiGrid(10, 10, torus=True)
        for i in range(n_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)

    def step(self):
        self.schedule.step()    

In [2]:
model = MyModel(10)
model.step()

8 activated
4 activated
7 activated
2 activated
9 activated
1 activated
5 activated
6 activated
0 activated
3 activated


scheludar method

https://mesa.readthedocs.io/en/stable/_modules/mesa/time.html

ANALYSIS MODULES

In [3]:
from mesa.datacollection import DataCollector

class MyAgent(Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name

    def step(self):
        print("{} activated".format(self.name))
        # Whatever else the agent does when activated

class MyModel(Model):      
    
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = BaseScheduler(self) #choose a schedular method
        self.grid = MultiGrid(10, 10, torus=True)
        for i in range(n_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)
        self.dc = DataCollector(model_reporters={"agent_count":
                                    lambda m: m.schedule.get_agent_count()},
                                agent_reporters={"name": lambda a: a.name})

    def step(self):
        self.schedule.step() 
        self.dc.collect(self)

In [108]:
model = MyModel(5)
for t in range(5):
    model.step()
model_df = model.dc.get_model_vars_dataframe() # sono uguali
agent_df = model.dc.get_agent_vars_dataframe() # questi due comandi?

0 activated
1 activated
2 activated
3 activated
4 activated
0 activated
1 activated
2 activated
3 activated
4 activated
0 activated
1 activated
2 activated
3 activated
4 activated
0 activated
1 activated
2 activated
3 activated
4 activated
0 activated
1 activated
2 activated
3 activated
4 activated


In [109]:
from mesa.batchrunner import BatchRunner

parameters = {"n_agents": range(1, 5)} # il numero di agenti va da uno a 20
batch_run = BatchRunner(MyModel, parameters, max_steps=3, # ogni ciclo a parametro fisso dura 10 step
                        model_reporters={"n_agents": lambda m: m.schedule.get_agent_count()})
batch_run.run_all()

4it [00:00, 961.39it/s]

0 activated
0 activated
0 activated
0 activated
1 activated
0 activated
1 activated
0 activated
1 activated
0 activated
1 activated
2 activated
0 activated
1 activated
2 activated
0 activated
1 activated
2 activated
0 activated
1 activated
2 activated
3 activated
0 activated
1 activated
2 activated
3 activated
0 activated
1 activated
2 activated
3 activated


In [100]:
batch_df = batch_run.get_model_vars_dataframe() #extract in a data frame

VISUALIZATION MODULES (doesn't work)

In [102]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    return portrayal

grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = ModularServer(MyModel,
                       [grid],
                       "My Model",
                       {'n_agents': 10})
server.launch()

AttributeError: module 'asyncio' has no attribute 'WindowsSelectorEventLoopPolicy'